<a href="https://colab.research.google.com/github/heminminmin/AI_Papering_Defects_Project/blob/main/base_open2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import pandas as pd
import numpy as np
import os
import re
import glob
import cv2

# pip3 install torch
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler

# pip3 install albumentations --user
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

# pip3 install torchvision
import torchvision.models as models

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from tqdm.auto import tqdm

import warnings
warnings.filterwarnings(action='ignore') 

In [ ]:
#!unzip -O cp949 '/content/drive/MyDrive/open.zip'

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
CFG = {
    'IMG_SIZE':224,
    'EPOCHS':10,
    'LEARNING_RATE':3e-4,
    'BATCH_SIZE':32,
    'SEED':41
}

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

In [ ]:
all_img_list = glob.glob('/content/open/train/*/*')
print('all_img_list :', all_img_list)

all_img_list : ['/content/open/train/들뜸/28.png', '/content/open/train/들뜸/38.png', '/content/open/train/들뜸/33.png', '/content/open/train/들뜸/31.png', '/content/open/train/들뜸/23.png', '/content/open/train/들뜸/30.png', '/content/open/train/들뜸/6.png', '/content/open/train/들뜸/27.png', '/content/open/train/들뜸/20.png', '/content/open/train/들뜸/35.png', '/content/open/train/들뜸/21.png', '/content/open/train/들뜸/46.png', '/content/open/train/들뜸/43.png', '/content/open/train/들뜸/44.png', '/content/open/train/들뜸/32.png', '/content/open/train/들뜸/41.png', '/content/open/train/들뜸/25.png', '/content/open/train/들뜸/37.png', '/content/open/train/들뜸/4.png', '/content/open/train/들뜸/10.png', '/content/open/train/들뜸/17.png', '/content/open/train/들뜸/8.png', '/content/open/train/들뜸/12.png', '/content/open/train/들뜸/14.png', '/content/open/train/들뜸/16.png', '/content/open/train/들뜸/0.png', '/content/open/train/들뜸/53.png', '/content/open/train/들뜸/11.png', '/content/open/train/들뜸/29.png', '/content/open/train/들뜸/19.png'

In [ ]:
df = pd.DataFrame(columns=['img_path', 'label'])
df['img_path'] = all_img_list

# 학습용 이미지 폴더 경로 수정
# df['label'] = df['img_path'].apply(lambda x : str(x).split('\\')[4])
df['label'] = df['img_path'].apply(lambda x : str(x).split('/')[4])
print('df :', df)

df :                             img_path label
0      /content/open/train/들뜸/28.png    들뜸
1      /content/open/train/들뜸/38.png    들뜸
2      /content/open/train/들뜸/33.png    들뜸
3      /content/open/train/들뜸/31.png    들뜸
4      /content/open/train/들뜸/23.png    들뜸
...                              ...   ...
1754  /content/open/train/가구수정/9.png  가구수정
1755  /content/open/train/가구수정/2.png  가구수정
1756  /content/open/train/가구수정/5.png  가구수정
1757  /content/open/train/가구수정/3.png  가구수정
1758  /content/open/train/가구수정/1.png  가구수정

[1759 rows x 2 columns]


In [ ]:
train, val, _, _ = train_test_split(df, df['label'], test_size=0.3, stratify=df['label'], random_state=CFG['SEED'])
print('train :', train)

train :                                 img_path   label
1633  /content/open/train/걸레받이수정/107.png  걸레받이수정
1566  /content/open/train/걸레받이수정/162.png  걸레받이수정
166        /content/open/train/오타공/9.png     오타공
1555  /content/open/train/걸레받이수정/163.png  걸레받이수정
305        /content/open/train/터짐/52.png      터짐
...                                  ...     ...
1349      /content/open/train/꼬임/172.png      꼬임
299       /content/open/train/터짐/159.png      터짐
505       /content/open/train/곰팡이/52.png     곰팡이
30         /content/open/train/들뜸/52.png      들뜸
1449     /content/open/train/몰딩수정/53.png    몰딩수정

[1231 rows x 2 columns]


In [ ]:
le = preprocessing.LabelEncoder()
train['label'] = le.fit_transform(train['label'])
val['label'] = le.transform(val['label'])
print('train :', train)

train :                                 img_path  label
1633  /content/open/train/걸레받이수정/107.png      1
1566  /content/open/train/걸레받이수정/162.png      1
166        /content/open/train/오타공/9.png     11
1555  /content/open/train/걸레받이수정/163.png      1
305        /content/open/train/터짐/52.png     15
...                                  ...    ...
1349      /content/open/train/꼬임/172.png      3
299       /content/open/train/터짐/159.png     15
505       /content/open/train/곰팡이/52.png      2
30         /content/open/train/들뜸/52.png      5
1449     /content/open/train/몰딩수정/53.png      7

[1231 rows x 2 columns]


In [ ]:
class CustomDataset(Dataset):
    def __init__(self, img_path_list, label_list, transforms=None):
        self.img_path_list = img_path_list
        self.label_list = label_list
        self.transforms = transforms
        
    def __getitem__(self, index):
        img_path = self.img_path_list[index]
        
        # TypeError: image must be numpy array type
        # issue: None => 경로에 한글명 존재
        # image = cv2.imread(img_path)
        # print('image :', image)

        # solved
        img_array = np.fromfile(img_path, np.uint8)
        image = cv2.imdecode(img_array, cv2.IMREAD_UNCHANGED)
        # print('image :', image)

        if self.transforms is not None:
            image = self.transforms(image=image)['image']
        
        if self.label_list is not None:
            label = self.label_list[index]
            return image, label
        else:
            return image
        
    def __len__(self):
        return len(self.img_path_list)

In [ ]:
train_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

test_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

In [ ]:
print('train[img_path].values :', train['img_path'].values)
print('train[label].values :', train['label'].values)
# print('train_transform :', train_transform)

train_dataset = CustomDataset(
    train['img_path'].values, train['label'].values, train_transform)

train_loader = DataLoader(
    train_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

val_dataset = CustomDataset(val['img_path'].values,
                            val['label'].values, test_transform)

val_loader = DataLoader(
    val_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

train[img_path].values : ['/content/open/train/걸레받이수정/107.png' '/content/open/train/걸레받이수정/162.png'
 '/content/open/train/오타공/9.png' ... '/content/open/train/곰팡이/52.png'
 '/content/open/train/들뜸/52.png' '/content/open/train/몰딩수정/53.png']
train[label].values : [ 1  1 11 ...  2  5  7]


In [ ]:
class BaseModel(nn.Module):
    def __init__(self, num_classes=len(le.classes_)):
        super(BaseModel, self).__init__()
        self.backbone = models.efficientnet_0(pretrained=True)
        self.classifier = nn.Linear(1000, num_classes)
        
    def forward(self, x):
        x = self.backbone(x)
        x = self.classifier(x)
        return x

In [ ]:
def validation(model, criterion, val_loader, device):
    
    model.eval()
    val_loss = []
    preds, true_labels = [], []

    with torch.no_grad():
        for imgs, labels in tqdm(iter(val_loader)):

            imgs = imgs.float().to(device)
            
            # RuntimeError: expected scalar type Long but found Int
            # labels = labels.to(device)

            # solved
            labels = labels.type(torch.LongTensor).to(device) 
            
            pred = model(imgs)
            
            loss = criterion(pred, labels)
            
            preds += pred.argmax(1).detach().cpu().numpy().tolist()
            true_labels += labels.detach().cpu().numpy().tolist()
            
            val_loss.append(loss.item())
        
        _val_loss = np.mean(val_loss)
        _val_score = f1_score(true_labels, preds, average='weighted')
    
    return _val_loss, _val_score

In [ ]:
def train(model, optimizer, train_loader, val_loader, scheduler, device):

    model.to(device)
    criterion = nn.CrossEntropyLoss().to(device)
    
    best_score = 0
    best_model = None
    
    for epoch in range(1, CFG['EPOCHS']+1):

        model.train()
        train_loss = []

        for imgs, labels in tqdm(iter(train_loader)):

            imgs = imgs.float().to(device)

            # RuntimeError: expected scalar type Long but found Int
            # labels = labels.to(device)

            # solved
            labels = labels.type(torch.LongTensor).to(device) 
            
            optimizer.zero_grad()
            
            output = model(imgs)
            loss = criterion(output, labels)
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
                    
        _val_loss, _val_score = validation(model, criterion, val_loader, device)
        _train_loss = np.mean(train_loss)
        
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val Weighted F1 Score : [{_val_score:.5f}]')
       
        if scheduler is not None:
            scheduler.step(_val_score)
            
        if best_score < _val_score:
            best_score = _val_score
            best_model = model
    
    return best_model

In [ ]:
model = BaseModel()
model.eval()
optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2, threshold_mode='abs', min_lr=1e-8, verbose=True)

infer_model = train(model, optimizer, train_loader, val_loader, scheduler, device)

  0%|          | 0/39 [00:00<?, ?it/s]

  0%|          | 0/17 [00:00<?, ?it/s]

Epoch [1], Train Loss : [1.73150] Val Loss : [1.13933] Val Weighted F1 Score : [0.64786]


  0%|          | 0/39 [00:00<?, ?it/s]

  0%|          | 0/17 [00:00<?, ?it/s]

...

  0%|          | 0/17 [00:00<?, ?it/s]

Epoch [9], Train Loss : [0.01633] Val Loss : [1.28016] Val Weighted F1 Score : [0.70780]


  0%|          | 0/39 [00:00<?, ?it/s]

  0%|          | 0/17 [00:00<?, ?it/s]

Epoch [10], Train Loss : [0.01937] Val Loss : [1.36420] Val Weighted F1 Score : [0.70002]


In [ ]:
test = pd.read_csv('/content/open/test.csv')


In [ ]:
test

,id,img_path
0,TEST_000,./test/000.png
1,TEST_001,./test/001.png
2,TEST_002,./test/002.png
3,TEST_003,./test/003.png
4,TEST_004,./test/004.png
...,...,...
787,TEST_787,./test/787.png
788,TEST_788,./test/788.png
789,TEST_789,./test/789.png
790,TEST_790,./test/790.png


In [28]:

test_img_paths = []
for i in test['img_path'].values:
    test_img_path = './open/' + i.replace('./', '')
    test_img_paths.append(test_img_path)
print('test_img_paths :', test_img_paths)
test_dataset = CustomDataset(test_img_paths, None, test_transform)

test_img_paths : ['./open/test/000.png', './open/test/001.png', './open/test/002.png', './open/test/003.png', './open/test/004.png', './open/test/005.png', './open/test/006.png', './open/test/007.png', './open/test/008.png', './open/test/009.png', './open/test/010.png', './open/test/011.png', './open/test/012.png', './open/test/013.png', './open/test/014.png', './open/test/015.png', './open/test/016.png', './open/test/017.png', './open/test/018.png', './open/test/019.png', './open/test/020.png', './open/test/021.png', './open/test/022.png', './open/test/023.png', './open/test/024.png', './open/test/025.png', './open/test/026.png', './open/test/027.png', './open/test/028.png', './open/test/029.png', './open/test/030.png', './open/test/031.png', './open/test/032.png', './open/test/033.png', './open/test/034.png', './open/test/035.png', './open/test/036.png', './open/test/037.png', './open/test/038.png', './open/test/039.png', './open/test/040.png', './open/test/041.png', './open/test/042

In [29]:

#test_dataset = CustomDataset(test['img_path'].values, None, test_transform)
test_loader = DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)


In [27]:
print(test['img_path'].values)

['./test/000.png' './test/001.png' './test/002.png' './test/003.png'
 './test/004.png' './test/005.png' './test/006.png' './test/007.png'
 './test/008.png' './test/009.png' './test/010.png' './test/011.png'
 './test/012.png' './test/013.png' './test/014.png' './test/015.png'
 './test/016.png' './test/017.png' './test/018.png' './test/019.png'
 './test/020.png' './test/021.png' './test/022.png' './test/023.png'
 './test/024.png' './test/025.png' './test/026.png' './test/027.png'
 './test/028.png' './test/029.png' './test/030.png' './test/031.png'
 './test/032.png' './test/033.png' './test/034.png' './test/035.png'
 './test/036.png' './test/037.png' './test/038.png' './test/039.png'
 './test/040.png' './test/041.png' './test/042.png' './test/043.png'
 './test/044.png' './test/045.png' './test/046.png' './test/047.png'
 './test/048.png' './test/049.png' './test/050.png' './test/051.png'
 './test/052.png' './test/053.png' './test/054.png' './test/055.png'
 './test/056.png' './test/057.png'

In [30]:
def inference(model, test_loader, device):
    model.eval()
    preds = []
    with torch.no_grad():
        for imgs in tqdm(iter(test_loader)):
            imgs = imgs.float().to(device)
            
            pred = model(imgs)
            
            preds += pred.argmax(1).detach().cpu().numpy().tolist()
    
    preds = le.inverse_transform(preds)
    return preds

In [31]:
preds = inference(infer_model, test_loader, device)

  0%|          | 0/25 [00:00<?, ?it/s]

In [33]:
submit = pd.read_csv('/content/open/sample_submission.csv')

In [34]:
submit['label'] = preds

In [35]:
submit.to_csv('./base_open2.csv', index=False)